# Spell Checker

In [ ]:
import string                       # python inbuild string class
import numpy as np                  # to handle matrix operations
import pandas as pd                 # to handle dataframes
import matplotlib.pylab as plt      # plotting diagrams
import nltk                         # NLTK package
from nltk.util import ngrams        # to get ngram from a given string
from collections import Counter     # to get frequency of word from a corpas
nltk.download("punkt")              # Essential package for NLTK

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Find Candidate Words

Dataset Used: The YAWL (Yet Another Word List) word list (263,533 words) by Peter Norvig

URL: https://norvig.com/ngrams/word.list

In [ ]:
# DICTIONARY
PATH = "drive/MyDrive/Datasets/word_list.txt"
with open(PATH, "r", encoding="utf-8") as f:
    dictionary = f.read().split("\n")

print(f"total number of lines: {len(dictionary)}")

total number of lines: 263534


In [ ]:
# ALPHABETS
alphabets = list("abcdefghijklmnopqrstuvwxyz")

In [ ]:
# FIND ALL CANDIDATES
def find_candidate_words(word):                     # get candidate words with single transformation
    ''' Find all candidate words for given misspelled word by using {insetion, deletion, transpose, substitution}
    Args:   word (string) = misspelled word
    Return: (list) a list containing all candidate words '''
    candidate_words = []
    for char in alphabets:
        for idx in range(len(word)+1):              # Insert candidate
            candidate_words.append(word[:idx]+char+word[idx:])
        for idx in range(len(word)):                # Substitution candidate
            candidate_words.append(word[:idx]+char+word[idx+1:])
    for idx in range(len(word)):                    # Deletion candidate
        candidate_words.append(word[:idx]+word[idx+1:])
    if(len(word)>1):                                # Transpose candidate
        for idx in range(len(word)-1):
            candidate_words.append(word[:idx]+word[idx+1]+word[idx]+word[idx+2:])
    return candidate_words

In [ ]:
# FIND VALID CANDIDATES
def find_valid_candidates(word):              # find all valid candidates from dictionary
    ''' Find all valid candidate words from a given word which also available in dictionary
    Args:   word = a string denoting misspelled word
    Return: A list of all valid candidate words '''
    candidate_first_edit = find_candidate_words(word)
    candidate_second_edit = []
    for _candidate_first_edit in candidate_first_edit:
        candidate_second_edit += find_candidate_words(_candidate_first_edit)
    candidates = set(candidate_first_edit).union(set(candidate_second_edit))
    return list(set(candidates).intersection(set(dictionary)))

In [ ]:
def find_valid_candidates(word):
    all_candidate_words = find_candidate_words(word)
    valid_candidates = list(set(all_candidate_words).intersection(set(dictionary)))
    return valid_candidates

In [ ]:
find_valid_candidates("brupt")

['brunt', 'abrupt', 'erupt', 'bruit', 'brut', 'brust']

## Minimum Edit distance

In [ ]:
def get_minimum_transformation(l):
    if l[0] <= l[1] and l[0] <= l[2]:
        return l[0],'d'
    elif l[1] <= l[2] and l[1] <= l[0]:
        return l[1],'i'
    else:
        return l[2],'s'

In [ ]:
def get_minimum_edit_distance(s1, s2):
    ''' Calculate the minimum edit distance between two strings
        Args:   s1 (string) = first string; error word
                s2 (string) = second string; candidate word
        Return: (int) minimum edit distance to transform from s1 to s2,
                (2D float array) calculate cost matrix '''

    m = len(s1)                         # length of source string
    n = len(s2)                         # length of target string
    mat = np.zeros((m+1, n+1))          # 2D array of dimention (n+1 X m+1) filled with 0's
    seq = [["" for a in range(n+1)] for _ in range(m+1)]

    # Initialization
    for i in range(1, m+1):             # Initializing row 1
        mat[i, 0] = mat[i-1, 0] + 1
        seq[i][0] = 'd'
    for j in range(1, n+1):             # Initializing column 1
        mat[0, j] = mat[0, j-1] + 1
        seq[0][j] = 'i'
    seq[0][0] = 'd'

    # Recursion
    for i in range(1, m+1):
        for j in range(1, n+1):
            sc = 2 if s1[i-1] != s2[j-1] else 0
            cost, transformation = get_minimum_transformation([mat[i-1, j] + 1,         # deletion
                                                               mat[i, j-1] + 1,         # insertion
                                                               mat[i-1, j-1] + sc])     # substitution
            mat[i, j] = cost
            seq[i][j] += transformation

    # get transformation sequence
    s1 = '#'+s1
    s2 = '#'+s2
    i,j = len(s1)-1, len(s2)-1
    sequence = list()
    while True:
        if seq[i][j] in ['d','i']:
            sequence.append(s1[i]+s2[j]+seq[i][j])
            if seq[i][j] == 'd':
                i -= 1
            elif seq[i][j] == 'i':
                j -= 1
        else:
            if s1[i] != s2[j]:
                sequence.append(s1[i]+s2[j]+seq[i][j])
            else:
                sequence.append('')
            i -= 1
            j -= 1
        if i == 0 and j == 0:   # stopping condition
            break

    # print(mat)
    # print(np.array(seq))

    return int(mat[m,n]), sequence

In [ ]:
# Test
get_minimum_edit_distance("intention", "execution")

(8,
 ['', '', '', '', 'nud', 'eui', 'eci', 'eei', 'exi', '', 't#d', 'n#d', 'i#d'])

In [ ]:
get_minimum_edit_distance('bleu', 'blue')

(2, ['ued', '', 'lui', '', ''])

## Find Prior and Channel probability



### Prior Model

In [ ]:
# Remove punchuation
import string
strip_chars = string.punctuation
def remove_punc(sentence):
    return sentence.translate(str.maketrans('', '', string.punctuation)).lower()

In [ ]:
remove_punc("Personal Story: Mayor Jerry Brown.")

'personal story mayor jerry brown'

In [ ]:
# GET TRAINING SET FOR TOKENS
PATH = "drive/MyDrive/Datasets/coca_sources.txt"
with open(PATH, 'r', encoding='utf8', errors='ignore') as f:
    sentences = [remove_punc(a.split("\t")[-1]).lower() for a in f.read().split("\n")[2:]]

# Test
sentences[:5]

['interview with byron york',
 'political headlines',
 'special report roundtable',
 'impact is the economy heading into a recession',
 'personal story mayor jerry brown']

In [ ]:
len(sentences)

189746

In [ ]:
# MAKE N-GRAM
def extract_ngrams(sentence, n):
    n_grams = ngrams(nltk.word_tokenize(sentence), n)
    # return [ ' '.join(grams) for grams in n_grams]
    return [grams for grams in n_grams]

In [ ]:
extract_ngrams("impact is the economy heading into a recession",3)

[('impact', 'is', 'the'),
 ('is', 'the', 'economy'),
 ('the', 'economy', 'heading'),
 ('economy', 'heading', 'into'),
 ('heading', 'into', 'a'),
 ('into', 'a', 'recession')]

In [ ]:
# COMPUTE ALL NGRAM FOR ENTIRE CORPUS
def get_all_ngrams(sentences, n):
    ngram_list = []
    for sentence in sentences:
        ngram_list += extract_ngrams(sentence, n)
    return ngram_list

In [ ]:
# AGGREGRATE NGRAMS
def calculateCounts(sentences, n):
    ngramList = get_all_ngrams(sentences, n)
    ngramCount = Counter(ngramList)
    return ngramCount

In [ ]:
unigram_count = calculateCounts(sentences, 1)  # unigram
bigram_count = calculateCounts(sentences, 2)   # bigram

In [ ]:
bigram_count[('there', 'is')]

60

In [ ]:
# GET PRIOR PROBABILITY
def get_prior_probability(word):
    return unigram_count[tuple(word.split())]/len(unigram_count)

In [ ]:
def get_bigram_probability(word):
    return bigram_count[tuple(word.split())]/len(bigram_count)

In [ ]:
# TEST
cases = ["to",
         "once",
         "there",
         "they"]
for a in cases:
    print(f"{a}\t:{get_prior_probability(a)}")

to	:0.32552125054356534
once	:0.0029981919300574463
there	:0.006122262147254709
they	:0.01360629849174925


### TODO: Creating Confusion Matrix

In [ ]:
# confusion matrix for insertion
with open("drive/MyDrive/Datasets/confusion_mats/add_mat.txt", 'r', encoding='utf8', errors='ignore') as f:
    lines = f.read().strip().split("\n")
ins_mat = [line.strip().split() for line in lines]

In [ ]:
# confusion matrix for deletion
with open("drive/MyDrive/Datasets/confusion_mats/del_mat.txt", 'r', encoding='utf8', errors='ignore') as f:
    lines = f.read().strip().split("\n")
del_mat = [line.strip().split() for line in lines]

In [ ]:
# confusion matrix for subtraction
with open("drive/MyDrive/Datasets/confusion_mats/sub_mat.txt", 'r', encoding='utf8', errors='ignore') as f:
    lines = f.read().strip().split("\n")
sub_mat = [line.strip().split() for line in lines]

In [ ]:
# confusion matrix for transpose
with open("drive/MyDrive/Datasets/confusion_mats/rev_mat.txt", 'r', encoding='utf8', errors='ignore') as f:
    lines = f.read().strip().split("\n")
trans_mat = [line.strip().split() for line in lines]

### Channel Model

In [ ]:
# COUNT MATRIX
def get_count(lexem):
    unigram_count = dict()
    bigram_count = dict()
    for sentence in sentences:
        bigrams = ["".join(a) for a in list(ngrams(lexem, n=2))]
        unigrams = ["".join(a) for a in list(ngrams(lexem, n=1))]
        bcount = Counter(bigrams)
        ucount = Counter(unigrams)

        for key in bcount:  # Count all bigrams
            try:
                bigram_count[key] += bcount[key]
            except:
                bigram_count[key] = bcount[key]

        for key in ucount:  # count all unigrams
            try:
                unigram_count[key] += ucount[key]
            except:
                unigram_count[key] = ucount[key]

    return unigram_count, bigram_count

In [ ]:
# TEST
a,b = get_count("have")
print(a)    # unigram's frequency
print(b)    # bigram's frequency

{'h': 189746, 'a': 189746, 'v': 189746, 'e': 189746}
{'ha': 189746, 'av': 189746, 've': 189746}


In [ ]:
# PROBAIBLITY OF CHANNEL
def char_probability(x, y, type):
    xi = (ord(x)-ord('a'))    # normalization
    yi = (ord(y)-ord('a'))
    uni_count, bi_count = get_count(x+y)
    if type.lower() == "d":
        return int(del_mat[xi][yi])/int(bi_count[x+y])
    elif type.lower() == "i":
        return int(ins_mat[xi][yi])/int(uni_count[y])
    elif type.lower() == "s":
        return int(sub_mat[xi][yi])/int(uni_count[y])
    elif type.lower() == "t":
        return int(trans_mat[xi][yi])/int(bi_count[x+y])
    return -1

In [ ]:
char_probability('h', 'a', 'd')

7.905304986666386e-05

In [ ]:
def get_channel_probability(word, candidate):
    edit_distance, edit_sequence = get_minimum_edit_distance(word, candidate)
    probability = 0
    for sequence in edit_sequence:
        if sequence != '':
            sequence_list = list(sequence)
            probability += char_probability(sequence_list[0], sequence_list[1], sequence_list[2])
    return probability

## Final Estimation

In [ ]:
# Final Estimation
def get_correct_word(misspelled_word, prev_word, next_word):
    probable_candidate_words = {}
    probable_bi_gram = {}
    candidates = find_valid_candidates(misspelled_word.lower())
    for candidate in candidates:
        prior_probability = get_prior_probability(candidate)
        channel_probability = get_channel_probability(misspelled_word, candidate)
        prob = prior_probability * channel_probability
        probable_candidate_words[candidate] = prob
    probable_candidate_words = sorted(probable_candidate_words.items(), key=lambda x: x[1], reverse=True)

    for candidate in probable_candidate_words:
        probable_bi_gram[prev_word+" "+candidate[0]+" "+next_word] = get_bigram_probability(prev_word+" "+candidate[0]) * get_bigram_probability(candidate[0]+" "+next_word)
    probable_bi_gram = sorted(probable_bi_gram.items(), key=lambda x: x[1], reverse=True)
    return probable_candidate_words, probable_bi_gram


In [ ]:
# Test suit
import pprint
test_cases = ["a goxd mine",
              "wear blxe shirt",
              "the nbw era",
              "skyroot rockt company",      # prediction error
              "the vintge computer"]
for cases in test_cases:
    print(f'Given: {cases}')
    words = cases.split()
    a, b = get_correct_word(words[1], words[0], words[2])
    print(f'Correct: {b[0][0]}')
    print("\nStatus:")
    pprint.pprint(a)
    pprint.pprint(b)
    print("=="*30)

Given: a goxd mine
Correct: a gold mine

Status:
[('good', 2.522383263240755e-06),
 ('gold', 1.0095805237818057e-07),
 ('gowd', 0.0),
 ('gox', 0.0),
 ('god', 0.0),
 ('goad', 0.0)]
[('a gold mine', 4.9295163515489077e-11),
 ('a good mine', 0.0),
 ('a gowd mine', 0.0),
 ('a gox mine', 0.0),
 ('a god mine', 0.0),
 ('a goad mine', 0.0)]
Given: wear blxe shirt
Correct: wear blue shirt

Status:
[('blue', 5.156459664476964e-08), ('blae', 0.0), ('blee', 0.0)]
[('wear blue shirt', 0.0), ('wear blae shirt', 0.0), ('wear blee shirt', 0.0)]
Given: the nbw era
Correct: the new era

Status:
[('new', 3.563179968570319e-05),
 ('now', 1.0697693746022501e-06),
 ('naw', 1.0252609859193965e-09)]
[('the new era', 9.751815722451626e-08),
 ('the now era', 0.0),
 ('the naw era', 0.0)]
Given: skyroot rockt company
Correct: skyroot rocks company

Status:
[('rocks', 4.824757580797159e-07),
 ('rocky', 2.2435122750706787e-08),
 ('rocket', 1.8454697746549136e-08),
 ('rock', 0.0)]
[('skyroot rocks company', 0.0),
 (